# Encoding classical data into qubits using Pennylane and Qiskit

In [1]:
import pennylane as qml
from pennylane import numpy as np

# Basis Encoding using PennyLane

In [2]:
# import Basis encoding template
from pennylane.templates.embeddings import BasisEmbedding

In [3]:
data = np.array([1,0,1])

dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit(data):
    BasisEmbedding(features=data,wires=range(3))
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2))

In [4]:
circuit(data)

(tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(-1., requires_grad=True))

# Angle encoding with Pennylane

In [5]:
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize
np.random.seed(42)

In [6]:
x, Y = load_iris().data, load_iris().target
x, Y = shuffle(x,Y)

x = x[:5]
Y = Y[:5]

In [7]:
print(x,Y)

[[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]] [1 0 2 1 1]


In [8]:
data = normalize(x)
print(data)

[[0.73659895 0.33811099 0.56754345 0.14490471]
 [0.8068282  0.53788547 0.24063297 0.04246464]
 [0.70600618 0.2383917  0.63265489 0.21088496]
 [0.73350949 0.35452959 0.55013212 0.18337737]
 [0.76467269 0.31486523 0.53976896 0.15743261]]


In [9]:
from pennylane.templates.embeddings import AngleEmbedding

In [10]:
num_qubits = len(data[0]) # No. of qubits= No. of features
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def circuit(data):
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    AngleEmbedding(features=data, wires=range(num_qubits), rotation='Y')    
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

In [11]:
circuit(data[0])

(tensor(-0.67177245, requires_grad=True),
 tensor(-0.33170563, requires_grad=True),
 tensor(-0.53756225, requires_grad=True),
 tensor(-0.14439814, requires_grad=True))

In [12]:
# Encode all data
@qml.qnode(dev)
def circuit(data):
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    for i in range(len(data)):
        AngleEmbedding(features=data[i], wires=range(num_qubits), rotation='Y')    
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

In [13]:
circuit(data)

(tensor(0.56960308, requires_grad=True),
 tensor(-0.97740396, requires_grad=True),
 tensor(-0.57357236, requires_grad=True),
 tensor(-0.67359663, requires_grad=True))

# Angle encoding with Qiskit

In [14]:
from qiskit import *
from qiskit.circuit.library import ZFeatureMap

In [15]:
featuremap_circ = ZFeatureMap(4, reps=1)

In [16]:
print(featuremap_circ.decompose())

     ┌───┐┌─────────────┐
q_0: ┤ H ├┤ P(2.0*x[0]) ├
     ├───┤├─────────────┤
q_1: ┤ H ├┤ P(2.0*x[1]) ├
     ├───┤├─────────────┤
q_2: ┤ H ├┤ P(2.0*x[2]) ├
     ├───┤├─────────────┤
q_3: ┤ H ├┤ P(2.0*x[3]) ├
     └───┘└─────────────┘


In [19]:
circ_data0 = featuremap_circ.assign_parameters(data[0]/2)

print(circ_data0.decompose())

     ┌───┐┌───────────┐ 
q_0: ┤ H ├┤ P(0.7366) ├─
     ├───┤├───────────┴┐
q_1: ┤ H ├┤ P(0.33811) ├
     ├───┤├────────────┤
q_2: ┤ H ├┤ P(0.56754) ├
     ├───┤├───────────┬┘
q_3: ┤ H ├┤ P(0.1449) ├─
     └───┘└───────────┘ 


In [23]:
circ_data1 = circ_data0.compose(featuremap_circ.assign_parameters(data[1]/2))

print(circ_data1.decompose())

     ┌───┐┌───────────┐ ┌───┐ ┌────────────┐
q_0: ┤ H ├┤ P(0.7366) ├─┤ H ├─┤ P(0.80683) ├
     ├───┤├───────────┴┐├───┤ ├────────────┤
q_1: ┤ H ├┤ P(0.33811) ├┤ H ├─┤ P(0.53789) ├
     ├───┤├────────────┤├───┤ ├────────────┤
q_2: ┤ H ├┤ P(0.56754) ├┤ H ├─┤ P(0.24063) ├
     ├───┤├───────────┬┘├───┤┌┴────────────┤
q_3: ┤ H ├┤ P(0.1449) ├─┤ H ├┤ P(0.042465) ├
     └───┘└───────────┘ └───┘└─────────────┘


In [25]:
circ = QuantumCircuit(4)

for i in range(len(data)):
    circ = circ.compose(featuremap_circ.assign_parameters(data[i]/2))

print(circ.decompose())

     ┌───┐┌───────────┐ ┌───┐ ┌────────────┐┌───┐┌────────────┐┌───┐»
q_0: ┤ H ├┤ P(0.7366) ├─┤ H ├─┤ P(0.80683) ├┤ H ├┤ P(0.70601) ├┤ H ├»
     ├───┤├───────────┴┐├───┤ ├────────────┤├───┤├────────────┤├───┤»
q_1: ┤ H ├┤ P(0.33811) ├┤ H ├─┤ P(0.53789) ├┤ H ├┤ P(0.23839) ├┤ H ├»
     ├───┤├────────────┤├───┤ ├────────────┤├───┤├────────────┤├───┤»
q_2: ┤ H ├┤ P(0.56754) ├┤ H ├─┤ P(0.24063) ├┤ H ├┤ P(0.63265) ├┤ H ├»
     ├───┤├───────────┬┘├───┤┌┴────────────┤├───┤├────────────┤├───┤»
q_3: ┤ H ├┤ P(0.1449) ├─┤ H ├┤ P(0.042465) ├┤ H ├┤ P(0.21088) ├┤ H ├»
     └───┘└───────────┘ └───┘└─────────────┘└───┘└────────────┘└───┘»
«     ┌────────────┐┌───┐┌────────────┐
«q_0: ┤ P(0.73351) ├┤ H ├┤ P(0.76467) ├
«     ├────────────┤├───┤├────────────┤
«q_1: ┤ P(0.35453) ├┤ H ├┤ P(0.31487) ├
«     ├────────────┤├───┤├────────────┤
«q_2: ┤ P(0.55013) ├┤ H ├┤ P(0.53977) ├
«     ├────────────┤├───┤├────────────┤
«q_3: ┤ P(0.18338) ├┤ H ├┤ P(0.15743) ├
«     └────────────┘└───┘└────────────┘


# Higher order feature encoding using Qiskit

In [26]:
from qiskit.circuit.library import ZZFeatureMap

In [27]:
featuremap_circ = ZZFeatureMap(2, reps=1)

In [28]:
print(featuremap_circ.decompose())

     ┌───┐┌─────────────┐                                          
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■──
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├
     └───┘└─────────────┘└───┘└──────────────────────────────┘└───┘
